# Projeto 1: Simulador Autónomo

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("⏳ A carregar dados e treinar modelos... Por favor aguarde.")

# ==============================================================================
# 1. CARREGAMENTO E PREPARAÇÃO DOS DADOS (Replicando a lógica do Data Prep)
# ==============================================================================
try:
    df = pd.read_csv('../assets/ScreenTime vs MentalWellness.csv')
    
    # Limpeza básica
    df_prep = df.drop('user_id', axis=1)
    
    # Feature Engineering
    df_prep['other_screen_hours'] = df_prep['screen_time_hours'] - (df_prep['work_screen_hours'] + df_prep['leisure_screen_hours'])
    df_prep['other_screen_hours'] = df_prep['other_screen_hours'].clip(lower=0)
    
    # Capping de Outliers (1% - 99%)
    cols_to_cap = ['age', 'screen_time_hours', 'work_screen_hours', 'leisure_screen_hours', 
                   'sleep_hours', 'productivity_0_100', 'exercise_minutes_per_week', 
                   'social_hours_per_week', 'other_screen_hours']
    for col in cols_to_cap:
        lower = df_prep[col].quantile(0.01)
        upper = df_prep[col].quantile(0.99)
        df_prep[col] = df_prep[col].clip(lower=lower, upper=upper)
        
    # Encoding
    df_encoded = pd.get_dummies(df_prep, columns=['gender', 'occupation', 'work_mode'], drop_first=True)
    
    # Identificar colunas de treino (X) e targets (y)
    target_cols = ['stress_level_0_10', 'sleep_quality_1_5', 'mental_wellness_index_0_100']
    features_cols = [col for col in df_encoded.columns if col not in target_cols]
    # Remover redundância se existir
    if 'screen_time_hours' in features_cols: features_cols.remove('screen_time_hours')
        
    # Scaling
    scaler = StandardScaler()
    X = df_encoded[features_cols]
    X_scaled = scaler.fit_transform(X)
    
    y_stress = df_encoded['stress_level_0_10']
    y_sleep = df_encoded['sleep_quality_1_5']
    
    # ==============================================================================
    # 2. TREINO DOS MODELOS (Usando os melhores algoritmos identificados)
    # ==============================================================================
    
    # Modelo de Stress: Random Forest (Geralmente melhor para padrões complexos)
    modelo_stress = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    modelo_stress.fit(X_scaled, y_stress)
    
    # Modelo de Sono: Linear Regression (Geralmente suficiente para esta relação)
    modelo_sono = LinearRegression()
    modelo_sono.fit(X_scaled, y_sleep)
    
    print("✅ Modelos treinados com sucesso!")
    print(f"   -> Stress Model R²: {modelo_stress.score(X_scaled, y_stress):.2f} (Treino)")
    print(f"   -> Sleep Model R²:  {modelo_sono.score(X_scaled, y_sleep):.2f} (Treino)")

except FileNotFoundError:
    print("❌ ERRO: O ficheiro 'ScreenTime vs MentalWellness.csv' não foi encontrado nesta pasta.")
    X, scaler, modelo_stress, modelo_sono, features_cols = None, None, None, None, None

# ==============================================================================
# 3. FUNÇÃO DE PREVISÃO INTERATIVA
# ==============================================================================

def fazer_previsao(age, gender, occupation, work_mode, 
                   screen_time, work_screen, leisure_screen, 
                   sleep_hours, productivity, exercise, social):
    
    if modelo_stress is None: return 0, 0

    # Criar DataFrame com os dados de input
    input_data = pd.DataFrame({
        'age': [age],
        'gender': [gender],
        'occupation': [occupation],
        'work_mode': [work_mode],
        'screen_time_hours': [screen_time],
        'work_screen_hours': [work_screen],
        'leisure_screen_hours': [leisure_screen],
        'sleep_hours': [sleep_hours],
        'productivity_0_100': [productivity],
        'exercise_minutes_per_week': [exercise],
        'social_hours_per_week': [social]
    })
    
    # Calcular Other Screen Hours
    input_data['other_screen_hours'] = input_data['screen_time_hours'] - (input_data['work_screen_hours'] + input_data['leisure_screen_hours'])
    input_data['other_screen_hours'] = input_data['other_screen_hours'].clip(lower=0)
    
    # One-Hot Encoding manual para garantir alinhamento
    input_encoded = pd.get_dummies(input_data, columns=['gender', 'occupation', 'work_mode'], drop_first=True)
    
    # Garantir que todas as colunas do treino existem no input (preencher com 0 se faltarem)
    for col in features_cols:
        if col not in input_encoded.columns:
            input_encoded[col] = 0
            
    # Reordenar colunas e Escalar
    input_final = input_encoded[features_cols]
    input_scaled = scaler.transform(input_final)
    
    # Prever
    pred_stress = modelo_stress.predict(input_scaled)[0]
    pred_sono = modelo_sono.predict(input_scaled)[0]
    
    return pred_stress, pred_sono

# ==============================================================================
# 4. INTERFACE GRÁFICA (WIDGETS)
# ==============================================================================

style = {'description_width': 'initial'}

w_age = widgets.IntSlider(value=30, min=18, max=70, description='Idade:', style=style)
w_gender = widgets.Dropdown(options=['Female', 'Male', 'Non-binary/Other'], value='Female', description='Género:', style=style)
w_occupation = widgets.Dropdown(options=['Employed', 'Student', 'Self-employed', 'Unemployed', 'Retired'], value='Employed', description='Ocupação:', style=style)
w_work_mode = widgets.Dropdown(options=['Hybrid', 'Remote', 'In-person'], value='Hybrid', description='Modo Trabalho:', style=style)

w_screen_total = widgets.FloatSlider(value=8.0, min=0, max=18, step=0.5, description='Horas Ecrã (Total):', style=style)
w_screen_work = widgets.FloatSlider(value=4.0, min=0, max=12, step=0.5, description='Horas Ecrã (Trabalho):', style=style)
w_screen_leisure = widgets.FloatSlider(value=2.0, min=0, max=12, step=0.5, description='Horas Ecrã (Lazer):', style=style)

w_sleep_hours = widgets.FloatSlider(value=7.0, min=3, max=10, step=0.5, description='Horas de Sono:', style=style)
w_productivity = widgets.IntSlider(value=50, min=0, max=100, description='Produtividade (0-100):', style=style)
w_exercise = widgets.IntSlider(value=120, min=0, max=500, description='Exercício (min/sem):', style=style)
w_social = widgets.IntSlider(value=5, min=0, max=30, description='Social (horas/sem):', style=style)

btn_calcular = widgets.Button(description="Calcular Previsão", button_style='primary', icon='calculator')
out_result = widgets.Output()

def on_click_calcular(b):
    with out_result:
        clear_output()
        if modelo_stress is not None:
            stress, sono = fazer_previsao(
                w_age.value, w_gender.value, w_occupation.value, w_work_mode.value,
                w_screen_total.value, w_screen_work.value, w_screen_leisure.value,
                w_sleep_hours.value, w_productivity.value, w_exercise.value, w_social.value
            )
            
            # Display visual
            print(f"\n📊 RESULTADOS DA SIMULAÇÃO:\n" + "-"*30)
            
            # Stress
            cor_stress = "🟢 Baixo" if stress < 4 else "🟠 Moderado" if stress < 7 else "🔴 ALTO"
            print(f"🧠 Stress Previsto (0-10):  {stress:.2f}  [{cor_stress}]")
            
            # Sono
            cor_sono = "🔴 Má" if sono < 2.5 else "🟠 Média" if sono < 4 else "🟢 Ótima"
            print(f"💤 Qualidade do Sono (1-5): {sono:.2f}  [{cor_sono}]")
            print("-"*30)
        else:
            print("Erro: Modelos não carregados.")

btn_calcular.on_click(on_click_calcular)

# Layout da Interface
ui = widgets.VBox([
    widgets.HTML("<h2>🏥 Simulador de Bem-Estar Mental</h2>"),
    widgets.HTML("<b>1. Perfil do Utilizador</b>"),
    widgets.HBox([w_age, w_gender]),
    widgets.HBox([w_occupation, w_work_mode]),
    widgets.HTML("<hr><b>2. Hábitos Digitais</b>"),
    w_screen_total,
    widgets.HBox([w_screen_work, w_screen_leisure]),
    widgets.HTML("<hr><b>3. Saúde e Estilo de Vida</b>"),
    widgets.HBox([w_sleep_hours, w_productivity]),
    widgets.HBox([w_exercise, w_social]),
    widgets.HTML("<br>"),
    btn_calcular,
    out_result
])

display(ui)

⏳ A carregar dados e treinar modelos... Por favor aguarde.
❌ ERRO: O ficheiro 'ScreenTime vs MentalWellness.csv' não foi encontrado nesta pasta.
